In [ ]:
import os
import shutil
from lammps_input import *

In [ ]:
cwd = os.getcwd()
sample_dir = os.path.join(cwd, 'sample')
trials_dir = os.path.join(cwd, 'Trials')

input_path = os.path.join(sample_dir, 'in.cond.sample')
data_path = os.path.join(sample_dir, 'lammps.data.sample')
qsub_path = os.path.join(sample_dir, 'lammps_qsub.sh.sample')

In [ ]:
masses = [[1, 39.948], [2, 20.1797]]

## Trial 2 - Half mass


In [ ]:
masses = [[1, 39.948], [2, 20.1797]]
trial2_dir = os.path.join(trials_dir, 'Trial2')
seeds = [123456, 123457, 123458, 123459, 123460]

with open(input_path, 'r') as sample_input:
    input_lines = sample_input.readlines()
    
with open(data_path, 'r') as sample_input:
    data_lines = sample_input.readlines()

if not os.path.exists(trial2_dir):
    os.mkdir(trial2_dir)

for i in range(5):
    run_dir = os.path.join(trial2_dir, 'Run%i' % i)
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    
    input_lines = change_seed(input_lines, seed=seeds[i])
    data_lines = change_masses(data_lines, masses)
    job_name = 'Trial2-Run%i' % i
    qsub_lines = lammps_qsub(qsub_path, name=job_name)
    
    inp = os.path.join(run_dir, 'in.cond')
    qs = os.path.join(run_dir, 'lammps_qsub.sh')
    dt = os.path.join(run_dir, 'lammps.data')
    
    export_lines(input_lines, inp)
    export_lines(data_lines, dt)
    export_lines(qsub_lines, qs)

## Trial 3 - Pair Coefficients

In [ ]:
seeds = [123456, 123457, 123458, 123459, 123460]
pair_coefficients = [0.01, 0.025, 0.05, 0.075, 0.1]

for trial_index, pc in enumerate(pair_coefficients):
    trial_dir = os.path.join(trials_dir, 'Trial3-%i' % (trial_index + 1))      # Initialize trial directory
    if not os.path.exists(trial_dir):
        os.mkdir(trial_dir)
        
    coefficient_list = [[1, 1, 0, 3.405],
                        [2, 2, 0, 3.405],
                        [1, 2, pc, 3.405]]
    
    with open(input_path, 'r') as sample_input:
        input_lines = sample_input.readlines()

    for i in range(5):
        run_dir = os.path.join(trial_dir, 'Run%i' % i)
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)

        input_lines = change_seed(input_lines, seed=seeds[i])
        input_lines = change_pair_coeff(input_lines, coefficient_list)
        job_name = 'PC%i-Run%i' % (pc, i)
        qsub_lines = lammps_qsub(qsub_path, name=job_name)

        inp = os.path.join(run_dir, 'in.cond')
        qs = os.path.join(run_dir, 'lammps_qsub.sh')
        dt = os.path.join(run_dir, 'lammps.data')

        export_lines(input_lines, inp)
        export_lines(qsub_lines, qs)
        shutil.copy(data_path, dt)